### ステップ1：環境準備

まず、このセルを実行して、分析に必要なツールをすべてインストールします。

**左の再生ボタン▶を押し、処理が終わるまで待ってください。**

In [ ]:
#@title 環境準備（ライブラリとフォントのインストール）
import os

print("--- 1. 必要なライブラリをインストールしています --- ")
# ✅ メッセージをすべて表示するために、エラーを隠す設定を削除しました
!pip install requests beautifulsoup4 janome wordcloud matplotlib

print("\n--- 2. 日本語フォントをダウンロードしています --- ")
font_path = '/content/NotoSansCJKjp-Regular.otf'
if not os.path.exists(font_path) or os.path.getsize(font_path) < 1024*1024: 
    !wget -O {font_path} https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/Japanese/NotoSansCJKjp-Regular.otf
else:
    print("フォントは既に存在します。")

print("\n✅ 環境準備が完了しました。次のセルに進んでください。")

### ステップ2：設定と実行

上のセルの実行が終わったら、ここにブログの情報を入力し、このセルを実行してください。

In [ ]:
#@title ◆ 設定と実行 ◆
#@markdown --- 
#@markdown ### 1. 分析したいブログの情報を入力
blog_url = "" #@param {type:"string"}
#@markdown 
#@markdown --- 
#@markdown ### 2. 分析する期間を指定
base_date_str = "" #@param {type:"date"}
days_to_go_back = 30 #@param {type:"slider", min:1, max:1000, step:1}

#@markdown --- 
#@markdown **設定が終わったら、このセルを実行してください**
import os
from datetime import datetime, timedelta, timezone
import requests
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from collections import Counter
import re
from urllib.parse import urljoin
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import time

if not blog_url:
    raise ValueError("ブログのURLが入力されていません。")

jst = timezone(timedelta(hours=9))
today = datetime.now(jst)
if base_date_str:
    end_date_obj = datetime.strptime(base_date_str, '%Y-%m-%d').replace(hour=23, minute=59, second=59, tzinfo=jst)
else:
    end_date_obj = today
start_date_obj = end_date_obj - timedelta(days=days_to_go_back)
font_path = '/content/NotoSansCJKjp-Regular.otf'
github_repo_url = "https://github.com/amufaamo/blog-to-tagcloud"

def load_stopwords(filepath='/content/stopwords.txt'):
    if not os.path.exists(filepath):
      print("stopwords.txtをダウンロードします。")
      stopwords_url = github_repo_url.replace('github.com', 'raw.githubusercontent.com') + '/main/stopwords.txt'
      !wget -q -O {filepath} {stopwords_url}
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = {line.strip() for line in f if line.strip()}
    print(f"{len(stopwords)}個の除外ワードを読み込みました。")
    return stopwords

def analyze_blog(base_url, start_date, end_date, stopwords):
    #... (この関数の内容は変更ありません) ...
    platform = detect_platform(base_url)
    if not platform: return None
    config = PLATFORM_CONFIGS[platform]
    all_text = ""
    current_url = base_url
    page_num = 1
    print(f"プラットフォーム: {platform.capitalize()} | 対象期間: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
    while current_url:
        print(f"記事一覧ページ {page_num} を取得中...")
        try:
            response = requests.get(current_url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            time.sleep(1)
        except:
            print("ページにアクセスできませんでした。")
            break
        posts = soup.select(config['post_container'])
        if not posts: break
        stop_crawling = False
        for post in posts:
            date_tag = post.select_one(config['date'])
            link_tag = post.select_one(config['permalink'])
            if not date_tag or not link_tag: continue
            post_date_str = date_tag.get(config['date_attribute'])
            if not post_date_str: continue
            try:
                post_date = datetime.fromisoformat(post_date_str.replace('Z', '+00:00'))
            except ValueError:
                continue
            if start_date <= post_date <= end_date:
                post_url = urljoin(base_url, link_tag['href'])
                try:
                    post_res = requests.get(post_url, headers={'User-Agent': 'Mozilla/5.0'})
                    post_res.encoding = 'utf-8'
                    post_soup = BeautifulSoup(post_res.text, 'html.parser')
                    content_body = post_soup.select_one(config['content_body'])
                    if content_body:
                        all_text += content_body.get_text() + "\n"
                    time.sleep(1)
                except:
                    pass
            elif post_date < start_date:
                stop_crawling = True
                break
        if stop_crawling:
            break
        if config['pagination']:
            next_page_tag = soup.select_one(config['pagination'])
            if next_page_tag and next_page_tag.has_attr('href'):
                current_url = urljoin(base_url, next_page_tag['href'])
                page_num += 1
            else:
                current_url = None
        else:
            current_url = None
    if not all_text: return None
    print("\nテキストの解析中...")
    t = Tokenizer()
    words = [token.surface for token in t.tokenize(all_text) 
             if token.surface not in stopwords and 
             token.part_of_speech.startswith(('名詞', '動詞', '形容詞')) and 
             len(token.surface) > 1 and not re.match(r'^[0-9a-zA-Z]+$', token.surface)]
    return Counter(words)

PLATFORM_CONFIGS = {
    'blogger': {'post_container': 'article.post-outer-container','permalink': 'h3.post-title a','date': 'time.published','date_attribute': 'datetime','content_body': 'div.post-body','pagination': 'a.blog-pager-older-link'},
    'hatenablog': {'post_container': 'article.entry','permalink': 'h1.entry-title a','date': 'time[datetime]','date_attribute': 'datetime','content_body': 'div.entry-content','pagination': 'a[rel="next"]'},
    'ameblo': {'post_container': 'li[data-unique-entry-id]','permalink': 'a[data-gtm-user-entry-title]','date': 'time','date_attribute': 'datetime','content_body': 'div[data-unique-entry-body]','pagination': 'a[data-gtm-button-name="記事一覧_次へ"]'},
    'note': {'post_container': 'div.o-cardNote','permalink': 'a.o-cardNote__link','date': 'time','date_attribute': 'datetime','content_body': 'div.note-common-styles__p','pagination': None}
}

def detect_platform(url):
    if 'hatenablog' in url: return 'hatenablog'
    if 'ameblo.jp' in url: return 'ameblo'
    if 'note.com' in url: return 'note'
    if 'blogspot.com' in url: return 'blogger'
    return None

# メイン処理
custom_stopwords = load_stopwords()
word_counter = analyze_blog(blog_url, start_date_obj, end_date_obj, custom_stopwords)

if word_counter:
    print("\n--- ✅単語の頻度ランキング TOP 50 ---")
    for word, count in word_counter.most_common(50):
        print(f"{word}: {count}回")
    print("\nワードクラウド画像を生成中...")
    if os.path.exists(font_path):
      try:
        wordcloud = WordCloud(width=1200, height=600, background_color='white', font_path=font_path, max_words=150, colormap='viridis').generate_from_frequencies(word_counter)
        plt.figure(figsize=(15, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
      except Exception as e:
        print(f"\nエラー: ワードクラウドの生成中にエラー: {e}")
    else:
        print(f"エラー：フォントファイルが見つかりません。ステップ1を再実行してください。")
else:
    print("エラー: 分析対象の記事が見つかりませんでした。")